# Loading & getting `MFLike` likelihood from `cobaya`

In this first tutorial, we will see how to instantiate `mflike` from `cobaya` interface and how we can retrieve the likelihood. This notebook is currently using the following modules: 

In [1]:
import matplotlib as mpl
import numpy as np

import camb
import cobaya

print("     Numpy :", np.__version__)
print("Matplotlib :", mpl.__version__)
print("      CAMB :", camb.__version__)
print("    Cobaya :", cobaya.__version__)

     Numpy : 1.26.4
Matplotlib : 3.8.3
      CAMB : 1.5.4
    Cobaya : 3.5


## Installing `MFLike` likelihood data

We first start by declaring the likelihood within a `dict` object. Most of the likelihood's options are set by default. Here we only set the input `sacc` file and the associated covariance file.

In [2]:
mflike_input_file = dict(
    input_file="LAT_simu_sacc_00044.fits", cov_Bbl_file="data_sacc_w_covar_and_Bbl.fits"
)
mflike_config = {"mflike.TTTEEE": mflike_input_file}

These files do not come with the `mflike` likelihood and we need to download and to install data needed by `mflike`. Here we will do it programatically in such a way we can use this notebook without any prerequisites. Code and data will be stored in the `/tmp` directory for the tutorial purpose but feel free to change it to whatever location you want. By the way the next command will take some times to proceed.

In [3]:
import os
import tempfile

from cobaya.install import install

packages_path = os.path.join(tempfile.gettempdir(), "LAT_packages")
install({"likelihood": mflike_config}, path=packages_path, no_set_global=True, skip_global=True);

[install] Installing external packages at '/tmp/LAT_packages'

likelihood:mflike.MFLike

[install] Checking if dependencies have already been installed...
[install] Check found no existing installation
[install] (If you expected this to be already installed, re-run `cobaya-install` with --debug to get more verbose output.)
[install] Installing...
[mflike] Downloading likelihood data file: https://portal.nersc.gov/cfs/sobs/users/MFLike_data/v0.8.tar.gz...


100%|███████████████████████████████████| 198M/198M [00:15<00:00, 13.8MiB/s]

[mflike] Downloaded filename v0.8.tar.gz


[mflike] Likelihood data downloaded and uncompressed correctly.
[install] Successfully installed! Let's check it...
[install] Installation check successful.

* Summary * 

[install] All requested components' dependencies correctly installed at /tmp/LAT_packages


## Loading `MFLike` *via* `cobaya`

Given a successful installation of the data, we then need to set the default values of CMB, foreground and nuisance parameters.

In [4]:
cosmo_params = {
    "cosmomc_theta": 0.0104092,
    "As": 1e-10 * np.exp(3.044),
    "ombh2": 0.02237,
    "omch2": 0.1200,
    "ns": 0.9649,
    "Alens": 1.0,
    "tau": 0.0544,
}

fg_params = {
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_p": 6.90,
    "beta_p": 2.20,
    "a_c": 4.90,
    "beta_c": 2.20,
    "a_s": 3.10,
    "T_d": 9.60,
    "a_gtt": 2.80,
    "a_gte": 0.10,
    "a_gee": 0.10,
    "a_psee": 0.003,
    "a_pste": 0.042,
    "xi": 0.10,
    "beta_s": -2.5,  # beta radio
    "alpha_s": 1,  # alpha radio
    "T_effd": 19.6,  # effective galactic dust temperature
    "beta_d": 1.5,  # beta galactic dust
    "alpha_dT": -0.6,  # galactic dust ell slope for T
    "alpha_dE": -0.4,  # galactic dust ell slope for E
    "alpha_tSZ": 0., # tSZ ell slope
    "alpha_p": 1,  # CIB poisson ell slope
}

nuisance_params = {
    # only ideal values for now
    "bandint_shift_LAT_93": 0,
    "bandint_shift_LAT_145": 0,
    "bandint_shift_LAT_225": 0,
    "calT_LAT_93": 1,
    "calE_LAT_93": 1,
    "calT_LAT_145": 1,
    "calE_LAT_145": 1,
    "calT_LAT_225": 1,
    "calE_LAT_225": 1,
    "cal_LAT_93": 1,
    "cal_LAT_145": 1,
    "cal_LAT_225": 1,
    "calG_all": 1,
    "alpha_LAT_93": 0,
    "alpha_LAT_145": 0,
    "alpha_LAT_225": 0,
}

We finally put everything into a dictionary to push it into `cobaya` configuration system. Here for the purpose of this tutorial, we use `camb` as Boltzmann solver with minimal settings. Another configuration `high_accuracy_settings` is also provided and you can switch within the `info` object.  
The likelihood also depends on the foreground model, the default model for which can also be obtained from mflike.BandpowerForeground.

In [5]:
minimal_settings = dict(lens_potential_accuracy=1)

high_accuracy_settings = dict(
    lens_potential_accuracy=8,
    lens_margin=2050,
    AccuracyBoost=2.0,
    lSampleBoost=2.0,
    lAccuracyBoost=2.0,
    kmax=10,
    k_per_logint=130,
    nonlinear=True,
    DoLateRadTruncation=False,
)

info = {
    "params": cosmo_params | fg_params | nuisance_params,
    "likelihood": mflike_config,
    "theory": {"camb": {"extra_args": minimal_settings},
               "mflike.BandpowerForeground": None},
    "packages_path": packages_path,
}

We will now get the model given the above parameters



In [6]:
from cobaya.model import get_model

model = get_model(info)

[camb] `camb` module loaded successfully from /home/garrido/Workdir/cmb/development/LAT_MFLike/pyenv/lib/python3.11/site-packages/camb
[mflike.mflike] Number of bins used: 3087
[mflike.mflike] Initialized!


To get a direct acces to the `MFLike` likelihood instance, we can retrieve it from the collection of
likelihoods of `cobaya` given its name (so far there is only our likelihood)



In [7]:
mflike = model.likelihood["mflike.TTTEEE"]

We can now play with the different data (Bbl, inverted covariance matrix) and we can also call the different likelihood methods.